In [1]:
import numpy as np
import cv2
import skimage.io 
import skimage.color
#Import necessary functions
from planarH import computeH_ransac, compositeH, computeH_norm
from matchPics import matchPics
from matplotlib import pyplot as plt

In [2]:
def loadVid(filename):
    cap = cv2.VideoCapture(filename)
    fps = cap.get(cv2.CAP_PROP_FPS)
    print(fps)
    frames = [] 
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
    cap.release()

    return frames

In [3]:
cv_cover = cv2.imread('../data/cv_cover.jpg') # (440, 350, 3)
book_frames = loadVid("../data/book.mov") # (480, 640) 641 frame
ar_frames = loadVid("../data/ar_source.mov") # (360, 640) 511 frame

30.0
25.0


### Test

## Code

In [4]:
from tqdm import tqdm
import time

In [10]:
vid_frames = []

for book_frame, ar_frame in zip(book_frames, tqdm(ar_frames)):

    cropped_ar = ar_frame[44:316, 320-108 : 320+108]

    orb = cv2.ORB_create(nfeatures=10000)

    kp1, des1 = orb.detectAndCompute(cv_cover, None)
    kp2, des2 = orb.detectAndCompute(book_frame, None)

    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

    matches_cv = bf.match(des1, des2)

    # Sort matches by distance
    matches_cv = sorted(matches_cv, key=lambda x: x.distance)

    num_matches = int(len(matches_cv) * 0.1)  # choose 10% of total matches
    best_matches = matches_cv[:num_matches]

    pts1_best = np.float32([kp1[m.queryIdx].pt for m in best_matches]).reshape(-1, 2)
    pts2_best = np.float32([kp2[m.trainIdx].pt for m in best_matches]).reshape(-1, 2)

    H2to1, inliers = computeH_ransac(pts2_best, pts1_best, 300, 2, adaptive = False)
    
    H2to1 = computeH_norm(pts2_best[inliers], pts1_best[inliers])
    
    h, w = book_frames[0].shape[:2]

    cropped_ar = cv2.resize(cropped_ar, dsize=(cv_cover.shape[1], cv_cover.shape[0]), interpolation=cv2.INTER_AREA)

    warped_cropped_ar = cv2.warpPerspective(cropped_ar, H2to1, (w, h))

    result = compositeH(H2to1, cropped_ar, book_frame)

    vid_frames.append(result)

100%|██████████| 511/511 [01:23<00:00,  6.15it/s]


In [12]:
def create_Video_io(imgList):
    writer = imageio.get_writer('../result/ar_2.avi', fps=25)

    for i in range(len(imgList)):
        img = imgList[i]
        writer.append_data(img)

    writer.close()
    
    return 0

In [13]:
import imageio
create_Video_io(vid_frames)

0